In [21]:
import sys
import numpy as np
import os, shutil
import re
import subprocess
import glob

# Overview
The workflow is divided into 3 steps

`Step 1`: preparation of hybrid structures and topologies.
Here, ligand maps from the literature are used: this ensures equivalent comparison to the earlier calculations.
Atom mapping between ligand pairs is generated with the script `atoms_to_morph.py` script.
Hybrid structure and topology is generated with `make_hybrid.py` script.
Further standard steps: placing system in the box, solvating, adding ions, creating energy minimization input.
This steps concludes with the energy minimization step (best to be run on the cluster).

`Step 2`: equilibrium runs.

`Step 3`: non-equilibrium transitions.

In [22]:
# here are some functions that are used in this workflow
def printInfo(runtype='em', run=1, case='xxx', edge='yyy_zzz', wp='water', state='stateA'):
    print('=' * 80)
    print(f'=== em{run:<5d} {case:8s} {edge:29s} {wp:12s} {state:12s} ===' )
    print('=' * 80)

# read edges from a file
def read_edges( path ):
    edgefile = path+'/edges.txt'
    fp = open(edgefile)
    out = {}
    for l in fp.readlines():
        l = l.rstrip()
        foo = l.split()
        key = 'edge_'+foo[0]+'_'+foo[-1]
        out[key] = ['lig_'+foo[0],'lig_'+foo[-1]]
    fp.close()
    return(out)


# create a folder
def create_folder( path, fname ):
    if not os.path.exists(path+'/'+fname):
        os.makedirs(path+'/'+fname)

# Choose System and Stage

In [34]:
# case to analyze.
case = 'jnk1'

In [35]:
targets = {}
with open(f'./input/targets.txt') as f:
    for line in f.readlines():
        line = line.split()
        targets[int(line[0])] = line[1]

for id, t in targets.items():
    if t == case:
        caseID = id
        case = t
        print(f'Target {caseID:02d}_{case} is chosen.')
        break
else:
    print('Target not found')

Target 01_jnk1 is chosen.


In [36]:
# forcefield
forcefield = 'smirnoff99Frosst-1.1.0.offxml'

# type of run. Can be one of 'em', 'eq', 'nvt', 'morphes'
runtype = 'em'

# all simulations will be done in 3 replicas
replicas = 3

# make all output verbose
bVerbose = True

# Set variables (paths, etc.)

In [37]:
# set paths relative to base
inputpath = './input'
mdppath = './mdppath'

# path where to find coordinates, topologies, hybrid topologies, simulations...
ligPath = f'systems/{caseID:02d}_{case}/03_docked/'
topPath = f'systems/{caseID:02d}_{case}/04_topo/{forcefield}/'
hybPath = f'systems/{caseID:02d}_{case}/05_hybrid/{forcefield}'

runpath =  {'em' : f'systems/{caseID:02d}_{case}/06_em/{forcefield}',
            'eq' : f'systems/{caseID:02d}_{case}/07_eq/{forcefield}',
            'nvt': f'systems/{caseID:02d}_{case}/08_nvt/{forcefield}',
            'morphes': f'systems/{caseID:02d}_{case}/09_morphes/{forcefield}'
           }

def getRunCoord(runtype, run=1, case='xxx', edge='yyy_zzz', wp='water', state='stateA'):
    if runtype == 'em':
        return f'{hybPath}/{wp}/{edge}/ions{run}.pdb'
    elif runtype == 'eq': 
        return f'{runpath["em"]}/{wp}/{edge}/{state}/em{run}/em{run}.gro'
    elif runtype == 'nvt':
        return f'{runpath["eq"]}/{wp}/{edge}/{state}/eq{run}/eq{run}.gro'
    elif runtype == 'morphes': 
        return f'{runpath["nvt"]}/{wp}/{edge}/{state}/nvt{run}/nvt{run}.gro'
    else:
        print('runtype not known')
        return ''

# for testing purposes, define one edge to be used later if needed
#------- TEST --------#
testedges = {}
testedges['edge_49585367_49137530'] = ['49585367', '49137530']
#------- TEST --------#

In [38]:
# read pre-defined edges
edgepath = f'{inputpath}/{caseID:02d}_{case}'
edges = read_edges( edgepath )
edges

{'edge_17124-1_18634-1': ['lig_17124-1', 'lig_18634-1'],
 'edge_18626-1_18624-1': ['lig_18626-1', 'lig_18624-1'],
 'edge_18636-1_18625-1': ['lig_18636-1', 'lig_18625-1'],
 'edge_18632-1_18624-1': ['lig_18632-1', 'lig_18624-1'],
 'edge_18635-1_18625-1': ['lig_18635-1', 'lig_18625-1'],
 'edge_18626-1_18658-1': ['lig_18626-1', 'lig_18658-1'],
 'edge_18639-1_18658-1': ['lig_18639-1', 'lig_18658-1'],
 'edge_18626-1_18625-1': ['lig_18626-1', 'lig_18625-1'],
 'edge_18638-1_18658-1': ['lig_18638-1', 'lig_18658-1'],
 'edge_18628-1_18624-1': ['lig_18628-1', 'lig_18624-1'],
 'edge_18631-1_18660-1': ['lig_18631-1', 'lig_18660-1'],
 'edge_18638-1_18634-1': ['lig_18638-1', 'lig_18634-1'],
 'edge_18626-1_18632-1': ['lig_18626-1', 'lig_18632-1'],
 'edge_18626-1_18630-1': ['lig_18626-1', 'lig_18630-1'],
 'edge_18631-1_18624-1': ['lig_18631-1', 'lig_18624-1'],
 'edge_18629-1_18627-1': ['lig_18629-1', 'lig_18627-1'],
 'edge_18634-1_18637-1': ['lig_18634-1', 'lig_18637-1'],
 'edge_18626-1_18627-1': ['lig_

In [39]:
# workpath/[water|protein]/edge* - every edge has its own folder
waterProtein = ['water','protein']
# workpath/[water|protein]/edge*/state[A|B] - two states will be considered for every edge
states = ['stateA','stateB']

# Create run files and submission scripts

In [40]:
################################################
# energy minimization #
################################################
    
# for testing set this variable to True
bTest = False

#----- TEST ------
edgesToUse = edges
if bTest==True:
    edgesToUse = testedges
#----- TEST ------    

bDeleteRunFiles = False
    
for edge in edgesToUse.keys():
    for wp in waterProtein:
        for state in states:
            # 3 replicas
            for run in range(1,replicas+1):
                printInfo(runtype=runtype, run=run, case=case, edge=edge, wp=wp, state=state)
                
                # create folder
                create_folder( f'{runpath[runtype]}/{wp}/{edge}/{state}/', f'{runtype}{run}/' )
                
                if bDeleteRunFiles:
                    # everything is deleted!
                    toclean = glob.glob(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/*.*')
                    for clean in toclean:
                        os.remove(clean)
                else:
                    if os.path.isfile(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/{runtype}{run}.log'):
                        # run log exists, so nothing is deleted
                        print('Energy minimization has been run already, nothing is deleted. Continue with next simulation.')
                        continue
                    else:
                        toclean = glob.glob(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/*.*')
                        for clean in toclean:
                            os.remove(clean)

                # specify input files
                mdp = f'{mdppath}/{runtype}_{state}.mdp'
                topology = f'{hybPath}/{wp}/{edge}/topol{run}.top'
                coord = getRunCoord(runtype, run=run, case=case, edge=edge, wp=wp, state=state)
                
                # specify output files
                tprfile = f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/{runtype}{run}.tpr' # temporary tpr file  
                mdout = f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/mdout.mdp'
                
                # call to gmx grompp
                process = subprocess.Popen(['gmx','grompp',
                                            '-p',topology,
                                            '-c',coord,
                                            '-o',tprfile,
                                            '-f',mdp,
                                            '-po',mdout,
                                            '-maxwarn',str(3)],
                                            stdout=subprocess.PIPE, 
                                            stderr=subprocess.PIPE)
                process.wait()    
            
                if bVerbose:
                    out = process.communicate()
                    print('STDOUT{} '.format(out[0].decode("utf-8")))
                    print('STDERR{} '.format(out[1].decode("utf-8")))
            
                # write submission file
                with open(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/{runtype}{run}.sh', 'w') as f:
                    f.write(f'#! /usr/bin/bash\n\
#$ -N {runtype}{run}_{case}_{wp}_{edge}_{state}\n\
#$ -S /bin/bash\n\
#$ -cwd\n\
#$ -j y\n\
#$ -pe smp 2\n\
#$ -q "*@wrhels4gk63*"\n\n\
source /home/dhahn3/bin/gromacs/gromacs-2019.4/bin/GMXRC\n\n\
WORKDIR=$(pwd)\n\
scp $WORKDIR/*.* $TMPDIR/\n\
cd $TMPDIR/\n\n\
/usr/lib64/openmpi/bin/mpirun -np 2 gmx_mpi mdrun -ntomp 8 -pin on -s {runtype}{run}.tpr -deffnm {runtype}{run}\n\n\
rsync -avzui $TMPDIR/ $WORKDIR/\n')

=== em1     jnk1     edge_17124-1_18634-1          water        stateA       ===
STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1963      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 40x40x40, spacing 0.110 0.110 0.110
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   C

STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1963      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 40x40x40, spacing 0.110 0.110 0.110
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        

Energy minimization has been run already, nothing is deleted. Continue with next simulation.
=== em3     jnk1     edge_17124-1_18634-1          protein      stateB       ===
Energy minimization has been run already, nothing is deleted. Continue with next simulation.
=== em1     jnk1     edge_18626-1_18624-1          water        stateA       ===
STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1712      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 36x36x36, spacing 0.116 0.116 0.116
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman 

STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1712      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 36x36x36, spacing 0.116 0.116 0.116
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        

STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1707      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 36x36x36, spacing 0.116 0.116 0.116
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        

...

STDOUTAnalysing residue names:
There are:   368    Protein residues
There are:     1      Other residues
There are: 22246      Water residues
There are:   137        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.105 0.105 0.105
This run will generate roughly 6 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     J

STDOUTAnalysing residue names:
There are:     1      Other residues
There are:  1704      Water residues
There are:    10        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 36x36x36, spacing 0.116 0.116 0.116
This run will generate roughly 0 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        

STDOUTAnalysing residue names:
There are:   368    Protein residues
There are:     1      Other residues
There are: 22252      Water residues
There are:   137        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.105 0.105 0.105
This run will generate roughly 6 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     J

STDOUTAnalysing residue names:
There are:   368    Protein residues
There are:     1      Other residues
There are: 22252      Water residues
There are:   137        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.105 0.105 0.105
This run will generate roughly 6 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     J

STDOUTAnalysing residue names:
There are:   368    Protein residues
There are:     1      Other residues
There are: 22252      Water residues
There are:   137        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.105 0.105 0.105
This run will generate roughly 6 Mb of data
 
STDERR                      :-) GROMACS - gmx grompp, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     J

# Check status of simulations

In [28]:
################################################
# check minimizations #
################################################

# for testing set this variable to True
bTest = False

#----- TEST ------
edgesToUse = edges
if bTest==True:
    edgesToUse = testedges
#----- TEST ------    
    
for edge in edgesToUse.keys():
    for wp in waterProtein:
        for state in states:
            # 3 replicas
            for run in range(1,replicas+1):
                printInfo(runtype=runtype, run=run, case=case, edge=edge, wp=wp, state=state)
                
                if os.path.isfile(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/{runtype}{run}.log'):
                    with open(f'{runpath[runtype]}/{wp}/{edge}/{state}/{runtype}{run}/{runtype}{run}.log') as f:
                        for line in f.readlines():
                            if re.search('Steepest Descents converged', line):
                                print(line.strip())
                                break
                        else:
                             print('\33[31mSimulation not (yet) converged.\33[0m')
                else:
                    print('\33[31mNo log file available, simulation probably not finished\33[0m')

=== em1     pde2     edge_49220392_49137530        water        stateA       ===
No log file available, simulation probably not finished
=== em2     pde2     edge_49220392_49137530        water        stateA       ===
No log file available, simulation probably not finished
=== em3     pde2     edge_49220392_49137530        water        stateA       ===
No log file available, simulation probably not finished
=== em1     pde2     edge_49220392_49137530        water        stateB       ===
No log file available, simulation probably not finished
=== em2     pde2     edge_49220392_49137530        water        stateB       ===
No log file available, simulation probably not finished
=== em3     pde2     edge_49220392_49137530        water        stateB       ===
No log file available, simulation probably not finished
=== em1     pde2     edge_49220392_49137530        protein      stateA       ===
No log file available, simulation probably not finished
=== em2     pde2     edge_49220392_491375